#Imports

In [10]:
import string
import re
import csv

import numpy as np
import pandas as pd

from joblib import dump, load


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer


from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score

# Instalación de librerias
import warnings
warnings.filterwarnings('ignore')

from scipy import stats as st
import sys
from pandas_profiling import ProfileReport
import re, unicodedata
import inflect
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, plot_precision_recall_curve
import matplotlib.pyplot as plt



#Preparación

In [11]:
pd.set_option('display.max_colwidth', None)

# Descargando las stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Función para tokenizar los tweets
def tokenizer(text):
    tt = TweetTokenizer()
    return tt.tokenize(text)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Carga de Datos

In [12]:
datos = pd.read_csv("/content/SuicidiosProyecto.csv",sep=",",index_col=0)
df_suicidio = datos.copy()
df_suicidio.shape

(195700, 2)

Tenemos 195.700 mensajes

In [13]:
df_suicidio.sample(5)

,text,class
Unnamed: 0,,
98724,Oh no I used to be funny cause I was depressed. And now I’m just depressed LOL,non-suicide
299067,"I know there’s a lot of you who feel the same way...I wanna die but I don’t want to at the same time… I don’t wanna be In pain but it’s what I really hate to feel but I like it... I really wish I could just end it right now and not have to worry for feel this way for another second but what’s stopping me? \n\nHell I don’t even know at this point but I keep on going… despite that Everything I’ve gone through.. I’m still alive and I say too people that no I’m gonna kms but I can’t do it... something is stopping me… \n\nWhy do I keep going? I mean it’s basically a form of torture and yet I’m okay with it and down play it? \n\nYeah I’m hurting and I hate myself but I don’t know what to do… I wanna cry… I personally have no one beside me, and if one day If I somehow I meet that special someone, I pray I won’t hurt them…\n\nTbh I don’t why I’m alive, I don’t wanna be but I keep on living.\n\nMuch love \n- some random person on Reddit",suicide
207883,"My grandfather is talking about the dems shipping (racist word for hispanics) to my rural WA town. Jesus christ he should just shut up now, he has gotten racist before but this is a new low.",non-suicide
116746,Who tryna give me da pics de thighs Filler filler filler Filler filler filler Filler filler filler Filler filler filler Filler filler filler Filler filler filler Filler filler filler pls send am desperate at horny hours,non-suicide
134795,"listening to the song streets by doja cat makes me feel sexy about myself as a girl listening to his amazing sensual song, I feel very powerful when I listen to it and like it’s a hot song 🥲",non-suicide


##Datos importantes

In [14]:
df_suicidio['class'].value_counts(dropna = False, normalize = True)

non-suicide    0.562928
suicide        0.437072
Name: class, dtype: float64

Proporción de mensajes categorizados com suicidas o no suicidas

## Limpieza

In [15]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
 
round1 = lambda x: clean_text_round1(x)
 
df_suicidio_clean = pd.DataFrame(df_suicidio.text.apply(round1))
 
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    text = re.sub(regex , ' ', text)
    # Eliminación de números
    text = re.sub("\d+", ' ', text)
    # Eliminación de espacios en blanco múltiples
    text = re.sub("\\s+", ' ', text)
    return text
 
round2 = lambda x: clean_text_round2(x)

 
df_suicidio_clean = pd.DataFrame(df_suicidio.text.apply(round2))


In [16]:
df_suicidio_clean['class'] = df_suicidio['class']
df_suicidio_clean.sample(5)

,text,class
Unnamed: 0,,
238851,I forgot to tell you that you re cute yesterday Im sorry 😔 you re so fucking cute,non-suicide
72262,I bought my rope A nice thick long rope to tie around my neck to suffocate my disgusting faggot shy good for nothing self I have no one to tell so here I am Not that anyone cares anyways I m not doing it today though but soon enough I just bought the rope,suicide
139327,Broken since birthFear anger and the withdrawal have been a part of my life since as long as I can remember I cannot relate to anyone due to my varying personality disorders anxiety adhd depression abandonment issues as well as battling the ego and trying to get any type of empathy when deep down I have no love to give just fear that is propped up by my false sense of ego Never really had any friends hated by all Wish I never existed that would be saved a lot of people that would rejoice in not seeing my moody aggresive self centred negative petty small minded twat I wouldnt wish this on anyone having to develop shields and close off since what is the point if one cannot and instinctively maybe doesnt want to get close to others and is wallowing in nostalgia only because it is in the past and therefore not a threat I feel like a cold dead monster if only I had the courage There are a lot of good people out there I am not one of them as I am too self centred and cold,suicide
65098,So tired I m just so fucking tired of being the one asking for advice asking for someone to talk to me asking to be told that I mean something I m tired of being incapable of being happy on my own I m tired of being dependant on people because of mental struggles I m tired of being me and I m tired of life Idk what to do and Idk why I m even typing this I m just so alone and I don t want to do this living thing anymore,non-suicide
100770,How am I supposed to tell a loved one with anxiety I need to be hospitalized In the past year Ive had four attempts that brought me to a hospital I learned coping skills DBT also a long time ago Im in therapy and on meds none of it is helping me I have a plan and Im just settling things before I enact it Its all getting worse and worse but Im feeling guilty about leaving my s o who has anxiety and depression I dont know how Im supposed to reach out to them when theyre already paranoid and anxious Ill have another attempt and leave them they tell me regularly theyre afraid of it I dont know how Im supposed to help myself I dont see any point in living anymore I cant stop thinking about carrying out my plan and I am so so tired How am I supposed to tell them when theyre already a wreck about my past attempts,suicide


In [17]:
df_suicidio_clean.to_csv('SuicidiosProyecto_clean.csv')

In [18]:
# El parámetro 'stratify' es útil para asegurar que ambos conjuntos de datos queden aproximadamente balanceados
# Cuando se cuenta con muy pocos datos etiquetados, por lo general se debe disponer de una menor cantidad de datos para evaluación
X_train, X_test, y_train, y_test = train_test_split(df_suicidio_clean['text'], df_suicidio_clean['class'], test_size = 0.2, stratify = df_suicidio_clean['class'], random_state = 1)

In [20]:
tfidf = TfidfVectorizer(tokenizer = tokenizer, stop_words = stop_words, lowercase = True)
X_tfidf = tfidf.fit_transform(X_train)
len(tfidf.vocabulary_)

128893

#Modelos

## Entrenando y evaluando un modelo usado TF-IDF

### Passive Aggressive Classifier

In [43]:
pre = [('initial',TfidfVectorizer(tokenizer = tokenizer, stop_words = stop_words, lowercase = True)),]

# Modelo
model = [('model', PassiveAggressiveClassifier(random_state=8))]

# Decalra el pipeline
pipeline = Pipeline(pre+model)

pipeline = pipeline.fit(X_train,y_train)

# Usamos la lbreria joblib
filename = 'pipeline.joblib'
# Se guarda
dump(pipeline, filename) 
# Se lee
tfidf_model = load(filename)

In [44]:
# tfidf_model = PassiveAggressiveClassifier(random_state=8)
#tfidf_model.fit(X_tfidf, y_train)

In [48]:
#y_train_tfidf_predict = tfidf_model.predict(X_tfidf)
y_train_tfidf_predict = tfidf_model.predict(X_train)
#y_test_tfidf_predict = tfidf_model.predict(tfidf.transform(X_test))
y_test_tfidf_predict = tfidf_model.predict(X_test)

In [49]:
ConfusionMatrixDisplay.from_predictions(y_train, y_train_tfidf_predict)

In [50]:
ConfusionMatrixDisplay.from_predictions(y_test, y_test_tfidf_predict)

In [51]:
if len(tfidf_model.classes_) == 2:
    print('Precision:', precision_score(y_train, y_train_tfidf_predict, pos_label = 'suicide'))
    print('Recall:', recall_score(y_train, y_train_tfidf_predict, pos_label = 'suicide'))
    print('F1:', f1_score(y_train, y_train_tfidf_predict, pos_label = 'suicide'))
else:
    # Para casos no binarios, es común calcular las métricas para cada clase
    print('Precision:', precision_score(y_train, y_train_tfidf_predict, average = None))
    print('Recall:', recall_score(y_train, y_train_tfidf_predict, average = None))
    print('F1:', f1_score(y_train, y_train_tfidf_predict, average = None))

Precision: 0.988148561526148
Recall: 0.983310925352195
F1: 0.9857238080588334


In [52]:
if len(tfidf_model.classes_) == 2:
    print('Precision:', precision_score(y_test, y_test_tfidf_predict, pos_label = 'suicide'))
    print('Recall:', recall_score(y_test, y_test_tfidf_predict, pos_label = 'suicide'))
    print('F1:', f1_score(y_test, y_test_tfidf_predict, pos_label = 'suicide'))
else:
    # Para casos no binarios, es común calcular las métricas para cada clase
    print('Precision:', precision_score(y_test, y_test_tfidf_predict, average = None))
    print('Recall:', recall_score(y_test, y_test_tfidf_predict, average = None))
    print('F1:', f1_score(y_test, y_test_tfidf_predict, average = None))

Precision: 0.9304443493673981
Recall: 0.8898696440053779
F1: 0.9097047926377435
